# 丸め誤差

In [1]:
s = 0
for i in range(1000):
    s += 0.001
s

1.0000000000000007

# 浮動小数点演算の収束条件の設定方法

In [2]:
eps = 1e-10
s = 0
while abs(s - 1.0) > eps:
    print(s)
    s += 0.1

0
0.1
0.2
0.30000000000000004
0.4
0.5
0.6
0.7
0.7999999999999999
0.8999999999999999


# 演算による桁落ち

微分

$$
\begin{align}
    f'(x) &= \lim_{h \to 0}\frac{f(x+h)-f(x)}{h} \\
    f(x) &= x^{2}のときのx=1で極限を計算してみる。\\
    &(f'(x)=2x, f'(1)=2)
\end{align}
$$

In [4]:
f = lambda x: x**2
df = lambda x,h: (f(x+h) - f(x)) / h
print(df(1, 0.001))
print(df(1, 1e-10))
print(df(1, 1e-15))
print(df(1, 1e-20)) 
# pythonのfloat型は10進数で約17桁なので、それ以上は誤差が生じる
# または、絶対値の近い数値の差分を取ることで、有効な桁が失われる桁落ちが発生

2.0009999999996975
2.000000165480742
2.220446049250313
0.0


解決策　→　関数の式がわかっているなら、数値微分はしない。関数の式はわかっているけど、複雑なので微分がめんどくさい場合は、シンボリックな計算ツール(e.g Sympy)を使う

In [5]:
import sympy as S
S.init_printing()
x = S.Symbol("x")
f = S.exp(x**3 * S.cos(S.log(x)**2))
print(f)

exp(x**3*cos(log(x)**2))


In [7]:
# 微分
S.diff(f)

10進数でfloat型が17桁程度の精度がある→0.001という数字は、
1.0000000000000000 x 10^(-3)(1の後に0が16個)くらいの桁まで正しい値で格納されていて、それ以下の桁に誤差が出る

### 計算の途中でこの17桁の精度が落ちる場合があるので、要注意！

２次方程式　$(x+2)(x+3)=x^{2}+5x+6$

In [9]:
import numpy as np

# 2次方程式を解く関数
def qeq(a, b, c):
    d = np.sqrt(b**2 - 4 * a * c)
    return ((-b + d) / (2 * a), (-b - d) / (2 * a))

In [10]:
qeq(1, 5, 6)

$(x+1)(x+0.000000001)=x^2+1.000000001x+0.000000001$になるはず。。。

In [12]:
# 数学上の答えは(-0.000000001, -1)
qeq(1, 1.000000001, 0.000000001)

0.0000000272292198は誤差

桁落ち発生！
$-b+\sqrt{b^{2}-4ac}$が0に近いときに起きる現象

$\sqrt{b^{2}-4ac}$に対して$b=-1.000000001$を足し算すると小数第8位まではきえてしまう。

In [9]:
a = np.sqrt(1.000000001**2 - 4 * 1 * 0.000000001)
a

0.999999999

In [14]:
minus_b = -1.000000001
minus_b + a # もともと17桁の有効数字が8桁までに桁落ちした。
# -0.000000002以降の桁数は信用できない。
# ↓ここに表示してある値。 コンピュータはもともと17桁の精度をもつから、
# 残りの9桁は適当な値(0とか)が勝手に入る！

-2.0000000544584395e-09

今回は $0.0000000544584395$は誤差

Wikipediaなどで調べると、一般的には正規化までを含めて桁落ちと定義しているようです。
つまり、浮動小数点演算で絶対値の近い値を減算→有効数字が減少→空いてしまったビットを0で埋める（正規化）という過程全体を、桁落ちと定義しています。

### 演算による桁落ちを回避するように計算式を工夫する
bの符号をチェックして先に桁落ちが発生しない式から計算して、もう一方を別の公式から求める

$ax^{2}+bx+c=0$の解を$\alpha,\beta$とすると

$$
\begin{align}
    \alpha &= \frac{-b-sign(b)\sqrt{b^{2}-4ac}}{2a} \\
    \beta &= \frac{c}{a\alpha}
\end{align}
$$

として解を導く $\alpha$の解放にsign(b)をいれることで、分子を必ず足し算にすることで、安定した解が求まる。不安定な解は別の方法で計算する

In [13]:
def qeq2(a, b, c):
    alpha = (-b - np.sign(b) * np.sqrt(b**2 - 4 * a * c)) / (2 * a) # 桁落ちが起きない工夫
    beta = c / (a * alpha)
    return (alpha, beta)

数値的に安定な解放

In [14]:
qeq2(1, 5, 6)

In [15]:
qeq2(1, 1.000000001, 0.000000001)

### その他の誤差として、”情報落ち”がある
これは、絶対値の大きな値と小さな値を加減算したときに、小さな値の数値が無視されてしまう現象

$$
  12345 \\
+)    0.42313 \\
  ----- \\
  12345.42313 \\
$$
絶対値の大きな数値と小さな数値で加減算を行ったとき、小さな数値が計算結果に反映されない

### 数値範囲の考慮

$$
    f(x) = log(1+e^{x}) \\
    x>>0のとき、f(x) \approx x
$$

In [16]:
# softplus関数
def softplus(x):
    return np.log(1+np.exp(x))

In [17]:
softplus(-1)

In [18]:
softplus(0)

In [19]:
softplus(1000)

/Users/inoueshinichi/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: RuntimeWarning: overflow encountered in exp
  This is separate from the ipykernel package so we can avoid doing imports until


$f(x)=log(1+e^{x})$が発散するのは$e^{x}$が発散するため

In [20]:
np.exp(1000)

/Users/inoueshinichi/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: RuntimeWarning: overflow encountered in exp
  """Entry point for launching an IPython kernel.


問題点は$f(x)=log(1+e^{x})$が数学的にPythonの有効数字に収まる範囲だが、計算途中のexp(x)でinfが発生していること

### 解決策は計算結果が同等になるように式変形を行い、infを回避する

$$
\begin{align}
    f(x) &= log{e^{x}(e^{-x}+1)} \\
    &= loge^{x} + log(e^{-x} + 1) \\
    &= x + log(e^{-x} + 1)
\end{align}
$$

In [39]:
def softplus2(x):
    return x + np.log(np.exp(-x) + 1)

softplus2(1000)

1000.0

今度はxが十分小さい時$e^{-x}$が発散する

In [41]:
softplus2(-1000)

/Users/inoueshinichi/.pyenv/versions/anaconda3-5.0.1/lib/python3.6/site-packages/ipykernel_launcher.py:2: RuntimeWarning: overflow encountered in exp
  


inf

### softplus + softplus2 = softplus3
x=0を境にして、計算が発散しないように異なる計算過程を組み合わせる

$$
    f(x) = \begin{cases}
        log(1+e^{x}) & (x >= 0) \\
        x + log(1+e^{-x}) & (x < 0)
    \end{cases}
$$

In [21]:
def softplus3(x):
    return max(0, x) + np.log(1 + np.exp(-abs(x)))

In [22]:
softplus3(-1)

In [23]:
softplus3(0)

In [24]:
softplus3(1000)

In [25]:
softplus3(-1000)

### 終了